In [1]:
import sys
import os

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# add parent directory to path\n",
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from paper_fetcher.country import CountryRecoder
from paper_fetcher.utils import recover_columns_names

In [3]:
FOLDER = '/home/luis/data/PubMedPapers/'

df_semi = pd.read_csv(
    os.path.join(FOLDER, 'publications_filtered_v7_semicolons.csv')
)
df_semi['Affiliations']=df_semi['Affiliations'].astype(str)
df_semi['Country'] = ''

df_semi

,Affiliations,PubMed ID,Author,Country
0,"Mackenzie Wearables Research Hub, Charles Perk...",38852004,Le Wei,
1,"School of Health Sciences, Faculty of Medicin...",38852004,Le Wei,
2,"Mackenzie Wearables Research Hub, Charles Perk...",38852004,Matthew N Ahmadi,
3,"School of Health Sciences, Faculty of Medicin...",38852004,Matthew N Ahmadi,
4,"School of Human Movement Nutrition Sciences, T...",38852004,Stewart Trost,
...,...,...,...,...
48691,"Chinese Academy of Medical Sciences, Beijing,...",22325671,L Li,
48692,"Nuffield Department of Obstetrics Gynaecology,...",25328660,Stephen H Kennedy,
48693,"World Endometriosis Research Foundation, Lond...",25328660,Stephen H Kennedy,
48694,"Nuffield Department of Obstetrics Gynaecology,...",25328660,Krina T Zondervan,


In [4]:
usa_locus = [
    "USA", "United States", "US", "Alabama", "California", "Albuquerque", "Ames IA", "Amherst MA", "Jackson MS", "Indianapolis",
    "Jacksonville FL", "Ann Arbor", "Albert Einstein College", "American Cancer Society", "Atlanta", "Massachusetts", "Mass.",
    "Boston", "Medford MA", "Milwaukee WI", "Arizona", "Baltimore", "Berkeley CA", "Bethesda MD", "Birmingham AL", "UC Davis",
    "Boulder CO", "Aurora CO", "Cambridge MA", "Harvard MA", "Brookline MA", "Brownsville TX", "Bronx NY", "New Haven CT", "Evanston, IL", "Evanston, Ill",
    "New Brunswick NJ", "Buffalo NY", "Burlington VT", "Califirnia", "New York", "Norwood MA", "Oakland", "Oregon", "Mountain View",
    "Palo Alto CA", "San Francisco CA", "Chapel Hill NC", "Charleston SC", "Charlestown MA", "Charlotte NC", "Dallas", "Rockville, MD",
    "Pennsylvania PA", "Pennsylvania", "Phoenix AZ", "Charlottesville VA", "Chicago", "Cincinnati", "Claremont CA", "Bethesda, Md",
    "Cleveland OH", "Collegeville PA", "Columbia SC", "Columbia University", "Providence RI", "Chevy Chase MD", "Bethesda, MD",
    "Puerto Rico", "Redwood City CA", "Triangle Park NC", "Columbus OH", "Corona CA", "Philadelphia", "Rochester MN", "Rochester, MN",
    "Rockville MD", "Sacramento CA", "Salt Lake City", "San Antonio TX", "Seattle", "Rhode Island", "Brooklyn College", "Portland, ME",
    "Brigham and Womens Hospital", "Calico", "Sioux Center IA", "CedarsSinai", "Los Angeles", "Colorado", "Danville PA", "San Diego, CA",
    "Davis CA", "Decatur GA", "Denver", "Stanford University", "Stanford Medical School", "Delaware", "West Virginia", "Mayo Clinic",
    "Louisville KY", "Brown University", "Blacksburg VA", "District of Columbia", "Downers Grove IL", "Duarte CA", "Indiana University",
    "Duke University", "Tennessee", "Texas", "East Lansing", "Emory University", "Englewood CO", "Evanston IL", "Rochester, Minn.",
    "George Washington University", "Icahn School", "Durham NC", "Farmington CT", "Florida", "Fort Collins CO", "Saint Louis University",
    "Fort Worth TX", "Framingham", "Frederick MD", "Fremont CA", "Gainesville", "Gaithersburg MD", "Grand Rapids MI", 
    "Glen Echo MD", "Hagerstown MD", "Hanover NH", "Harrison NJ", "Hartford CT", "Hawaii", "Mississippi", "Michigan", "San Francisco VA",
    "Miami", "Utah", "Honolulu", "Houston", "Illinois", "Iowa", "Irvine CA", "Ithaca NY", "Johns Hopkins", "Kalamazoo MI",
    "Kansas", "Kentucky", "Prussia PA", "La Jolla", "Lancaster PA", "Las Vegas NV", "Lewisburg PA", "Lexington KY", " Murdoch Children's Research Institute ",
    "Lisle IL", "Pittsburgh PA", "Yale Universit", "Yale School of Medicine", "Wisconsin", "Winston Salem NC", " Alvin J. Siteman Cancer Center",
    "WinstonSalem NC", "Washington", "Waco TX", "Virginia", "Vermont", "Vanderbilt", "Maryland", "North Carolina", "James J. Peters Veterans Affairs",
    "Pittsburgh", "Tulane", "Tucson", "Torrance", "Wake Forest School of Medicine", "VA Palo Alto", "VA San Diego", "National Institute on Aging",
    "Tarrytown NY", "Long Beach CA", "Louisiana", "Madison WI", "Masschusetts", "Memphis TN", "Charles Bronfman Institute",
    "Harvard", "Lerner Research Institute", "Minneapolis", "Minnesota", "Missouri", "Moffitt Cancer Center", "New Haven, CT",
    "Morristown NJ", "Mountain View CA", "Nashville", "Marina del Rey", "Maywood IL", "South Carolina", "Tampa FL", "Providence, RI",
    "Monell Chemical Senses Center", "Blood Institute MA", "Taussig Cancer Institute", "Tallahassee", "Syracuse NY", "East Carolina University",
    "Redwood City", "Oklahoma", "Ohio", "Palo Alto", "Portland OR", "New Hyde Park NY", "New Hampshire", "New Orleans", "Hagerstown, MD",
    "New Jersey", "North Haven CT", "Northeastern University", "UAB Lung Imaging Lab", "Stony Brook NY", "Stamford CT", " Dana-Farber Cancer Institute",
    "Stanford", "St Louis MO", "Connecticut", "ChanZuckerberg Biohub", "Cleveland", "Greenville NC", "San Diego CA", "Brigham Women's Hospital",
    "Demoulas Center for Cardiac Arrhythmias", "Portland ME", "SUNY Downstate College", "SWOG Statistical Center", "Broad Institute",
    "Santa Cruz CA", "Scottsdale AZ", "Scripps", "Severn Health Solutions", "Severna Park MD", "Frank H. Netter MD School of Medicine",
    "NIH ENIGMA Center for Worldwide Medicine"
]

In [5]:
arg_locus = ["Argentina", "Buenos Aires"]
aus_locus = ["Australia", "Melbourne", "Perth", "Cancer Council Victoria", "Curtin University", "Sydney", "Queensland", "Monash Health", "Tasmania",
             "Monash University", "Australi", "Royal Victorian Eye and Ear Hospital", "University of New South Wales", "Peninsula Health",
             "National Centre for Healthy Ageing", "NHMRC"]
aut_locus = ["Austria", "Graz", "Innsbruck"]

bah_locus = ["Bahrain"]
ban_locus = ["Bangladesh", "Dhaka"]
bar_locus = ["Barbados"]
bel_locus = ["Belgium", "Leuven"]
blr_locus = ["Belarus"]
bos_locus = ["Bosnia and Herzegovina", "Sarajevo", "Bosnia", "Bosnia Herzegovina"]
bra_locus = ["Brazil", "Brasil"]
bru_locus = ["Brunei"]

cam_locus = ["Cameroon", "Cameroun"]
can_locus = ["Canada", "British Columbia", "Ontario", "Québec", "Quebec", "Toronto", "Vancouver", "Lady Davis Institute",
             "Montreal", "Hamilton Health Sciences", "McGill University", "Sunnybrook Research Institute", "Thrombosis Atherosclerosis Research Institute"]
chi_locus = ["Chile", "Talca"]
chn_locus = ["China", "Beijing", "CHINA", "Chinese", "Central South University", "Changping", "Chengdu", "Dongfang",
             "Xian Jiaotong", "Tongxiang", "Fudan University", "Guangzhou", "Guangdong", "Harbin", "Heilongjiang", "Sun Yat-sen University",
             "Furong", "Guangxi", "Hong Kong", "Hongqiao", "Dongguan", "Hunan", "Jiangsu", "Jinan", "Zhengzhou", "NHC Key Laboratory of Cardiovascular Molecular Biology",
             "Zhongshan", "Liaoning", "Liuzhou", "Zhejiang", "Yichang", "Xiangya", "Wuhan", "Tianjin", "Tsinghua",
             "Wannan Medical College", "Nanchang", "Shandong", "Li Ka Shing Faculty of Medicine", "Xuanwu", "Tongji",
             "Soochow University", "Oujiang", "Peking", "Pengzhou", "Yueyang", "Yinchuan", "Yantai", "Fujian", "Key Laboratory of Molecular Cardiovascular Sciences",
             "Sun Yatsen University", "Shiyan", "Shidong", "Shenzhen", "Shengjing", "Guangan", "Huanggang", "Hubei", "Emergency Medicine Difficult Diseases Institute",
             "Huashan", "Nanjing", "Shaanxi", "Shanghai", "Southern Medical University", " State Key Laboratory of Organ Failure Research",
             "National Clinical Research Center for Kidney Disease", "Department of Neurology, Second Affiliated Hospital"]
civ_locus = ["Côte dIvoire"]
col_locus = ["Colombia", "Bogotá", "Bogota"]
cro_locus = ["Croatia"]
cyp_locus = ["Cyprus"]
cz_locus  = ["Czech", "Prague"]

den_locus = ["Denmark", "Aarhus", "Danish", "Danmark", "Faroe Islands", "Greenland", "Herlev and Gentofte Hospital",
             "Rigshospitalet", "Copenhagen"]
dom_locus = ["Dominican Republic", "Dominicana"]

egy_locus = ["Egypt"]
esp_locus = ["Spain", "España",  "Barcelona", "Madrid", "Ramón y Cajal", "Asturias", "Pamplona", "Granada", "BCNAIM", "Alberto Sols", "IRYCIS",
             "GRINCAVA", "Murcia", "Oviedo", "Bellvitge Biomedical Research Institute", "CIBERESP", "Gipuzkua", "Sevilla", "Gipuzkoa", "Barakaldo",
             "Alcalá de Henares"]
est_locus = ["Estonia", "Tartu"]
eth_locus = ["Ethiopia"]

fin_locus = ["Finland", "Oulu", "FINLAND", "Finlan", "Helsinki", "Kuopio", "Finnish", "Pirkanmaa", "University of Tampere"]
fra_locus = ["France", "Marseille", "Bordeaux", "Paris", "Inserm", "Centre de Recherche des Cordeliers", "Créteil", " Laboratoire d'Excellence GR-Ex",
             "Dijon", "Lille", "Nantes", "Montpellier", "la Santé", "Lyon", "Villejuif", "CNRS", "Toulouse", " CHRU Nancy", "INSERM"]

gam_locus = ["Gambia"]
ger_locus = ["Germany", "Heidelberg", "Augsburg", "Berlin", "Munich", "Deutschland", "Essen", "Freiburg", "Fraunhofer", "Leipzig",
             "German", "Greifswald", "Helmholtz", "Erlangen", "Hamburg", "DKFZ", "DZHK", "LIFE Child", "MaxPlanck",
             "Neuherberg", "CharitéUniversität", "FriedrichAlexanderUniversität", "HumboldtUniversität", 
             "LudwigMaximilian", "Köln", "Cologne"]
gha_locus = ["Ghana"]
gre_locus = ["Greece", "University of Crete", "University of Athens"]

hun_locus = ["Hungary", "Budapest"]

ice_locus = ["Iceland", "Kopavogur"]
ind_locus = ["Kolkata"]
ina_locus = ["Indonesia"]
ire_locus = ["Ireland", "Dublin"]
irn_locus = ["Iran"]
irq_locus = ["Iraq"]
isr_locus = ["Israel"]
ita_locus = ["Italy", "Ragusa","Sassari", "Rome", "Bergamo", "Roma", "DIBRIS", "Torino", "Florence", "Fondazione IRCCS", "Humanitas University",
             "ITALY", "Italia", "Itlay", "Milan", "Padua", "Pavia", "Università Cattolica Sacro Cuore", "Udine", "Padova", "Catanzaro"]

jam_locus = ["Jamaica"]
jpn_locus = ["Japan", "Tokyo", "Osaka", "Chiba University", "Tokushima", "Tohoku", "Fukuoka", "Hiroshima",
             "Iwate Medical University", "Kanagawa", "Nagoya", "Gifu", "Otsu", "RIKEN Center", "Saga University"]

ken_locus = ["Kenya", "Nairobi"]
ksa_locus = ["Saudi Arabia", "Riyadh"]
kuw_locus = ["Kuwait"]

lao_locus = ["Laos"]
lie_locus = ["Liechtenstein"]
lth_locus = ["Lithuania"]
lux_locus = ["Luxembourg"]

mac_locus = ["Macedonia"]
mal_locus = ["Mali"]
mex_locus = ["Mexico", "México"]
mlw_locus = ["Malawi"]
mly_locus = ["Malaysia"]
mzb_locus = ["Mozambique"]

nam_locus = ["Namibia", "Windhoek"]
nep_locus = ["Nepal"]
nig_locus = ["Nigeria", "Abuja", "Lagos"]
nor_locus = ["Norway", "Oslo", "Bergen", "Norwegian", "Trondheim"]
nth_locus = ["Netherlands", "Amsterdam", "EMGO", "Vrije Universiteit", "Leiden", "Maastricht", "Utrecht", "Hubrecht", "Sanquin Diagnostic Services",
             "GRIAC", "Julius Center for Health Sciences", "VU University Medical Center", "Rotterdam", "Nederland",
             "Bilthoven", "Radboud", "Delft University", "Rijnstate", "Julius Center"]
nz_locus  = ["New Zealand", "Auckland"]

pak_locus = ["Pakistan"]
pan_locus = ["Panama"]
per_locus = ["Perú", "Peru"]
phi_locus = ["Philippines"]
pol_locus = ["Poland"]
por_locus = ["Portugal", "Aveiro", "Coimbra"]

qat_locus = ["Qatar", "Doha"]

rom_locus = ["Romania", "Ana Aslan International Foundation"]
rsa_locus = ["South Africa", "University of the Witwatersrand", "Cape Town"]
rus_locus = ["Russia", "Moscow"]
rwa_locus = ["Rwanda"]

sen_locus = ["Senegal"]
sin_locus = ["Singapore", "Nanyang"]
skr_locus = ["South Korea", "Republic of Korea", "Busan", "Seoul", "Cheongjusi", "Chungnam", "Daegu", "Daejeon", "Goyangsi",
             "Gwangju", "Gyeonggido", "Ulsan", "Hwasun", "Korea University", "Uijeongbu Eulji", "Yonsei", "South of Korea",
             "Seongnam", "Gyeonggi-do", "Samsung Medical Center"]
sle_locus = ["Sierra Leone"]
slk_locus = ["Slovak"]
slv_locus = ["Slovenia"]
srb_locus = ["Serbia"]
swe_locus = ["Sweden", "Stockholm", "Malmö", "Lund", "Umeå", "Karolinska Institute", "Linköping", "Uppsala", "Sweded",
             "Karolinska"]
swi_locus = ["Switzerland", "Botnar Research Centre", "Zurich", "Bern", "Geneva", "Swiss", "Lausanne"]

tai_locus = ["Taiwan", "Taipei", "Chung Shan Medical", "Taichung", "Hualien", "Kaohsiung", "Taoyuan", "Tamkang", "Hsinchu",
             "National YangMing University"]
tan_locus = ["Tanzania", "Dar es Salaam"]
tha_locus = ["Thailand"]
tur_locus = ["Turkey"]

uae_locus = ["UAE", "United Arab Emirates", "Abu Dhabi"]
uga_locus = ["Uganda"]
uk_locus  = ["United Kingdom", "UK", "Aberdeen", "Addenbrookes Hospital", "N Ireland", "Scotland", "Belfast", "Bristol", "Novo Nordisk Research Centre",
            "Cardiff", "London", "NHS Trust", "NHS trust", "British", "Edinburgh", "Glasgow", "NHS Foundation Trust", "National Problem Gambling Clinic",
            "John Radcliffe Hospital", "Lancaster University", "Leeds", "Leicester", "Liverpool", "United ingdom", "Oxford Centre for Computational Neuroscience",
            "United kingdom", "Manchester", "UNITED KINGDOM", "Wellcome Centre for Human Genetics", "Lothian", "University of Warwick", 
            "Wellcome Trust Centre for Human Genetics", "Loughborough University", "Ormond Street Hospital", "NIHR", "Alderley Park", "NHS Dumfries Galloway",
            "MRC Integrative Epidemiology", "MRC Population Health Research", "MRCBHF", "MRCPHE", "University of Exeter", "University of Nottingham",
            "Medical Research Council", "BHF Data Science Centre", "BRC Oxford Haematology", "University of Southampton", "Centre for Drug Safety Science",
            "Francis Crick Institute", "Robertson Center for Biostatistics", "Midlothian", "Worcestershire", "Newcastle University", "Middlesex",
            "UCL Institute of Child Health", "Swansea University", "Gibraltar", "Nuffield", "University of Cambridge", "Royal National Hospital for Rheumatic Diseases",
            "University of Oxford", "U.K.", "UCL Queen Square Institute of Neurology", "University of Birmingham", "Mount Vernon Hospital"]
ukr_locus = ["Ukraine"]
uru_locus = ["Uruguay", "Montevideo"]
uzb_locus = ["Uzbekistan"]

zim_locus = ["Zimbabwe"]

In [6]:
country_pipe = Pipeline([
    ('usa', CountryRecoder(country_name="USA", aliases=usa_locus).set_output(transform='pandas')),
    ('uk', CountryRecoder(country_name="UK", aliases=uk_locus).set_output(transform='pandas')),
    ('ire', CountryRecoder(country_name="Ireland", aliases=ire_locus).set_output(transform='pandas')),
    ('bra', CountryRecoder(country_name="Brazil", aliases=bra_locus).set_output(transform='pandas')),
    ('gre', CountryRecoder(country_name="Greece", aliases=gre_locus).set_output(transform='pandas')),
    ('chn', CountryRecoder(country_name="China", aliases=chn_locus).set_output(transform='pandas')),
    ('aus', CountryRecoder(country_name="Australia", aliases=aus_locus).set_output(transform='pandas')),
    ('nz', CountryRecoder(country_name="New Zealand", aliases=nz_locus).set_output(transform='pandas')),
    ('can', CountryRecoder(country_name="Canada", aliases=can_locus).set_output(transform='pandas')),
    ('esp', CountryRecoder(country_name="Spain", aliases=esp_locus).set_output(transform='pandas')),
    ('por', CountryRecoder(country_name="Portugal", aliases=por_locus).set_output(transform='pandas')),
    ('fra', CountryRecoder(country_name="France", aliases=fra_locus).set_output(transform='pandas')),
    ('bel', CountryRecoder(country_name="Belgium", aliases=bel_locus).set_output(transform='pandas')),
    ('ger', CountryRecoder(country_name="Germany", aliases=ger_locus).set_output(transform='pandas')),
    ('aut', CountryRecoder(country_name="Austria", aliases=aut_locus).set_output(transform='pandas')),
    ('ita', CountryRecoder(country_name="Italy", aliases=ita_locus).set_output(transform='pandas')),
    ('net', CountryRecoder(country_name="Netherlands", aliases=nth_locus).set_output(transform='pandas')),
    ('pol', CountryRecoder(country_name="Poland", aliases=pol_locus).set_output(transform='pandas')),
    ('swi', CountryRecoder(country_name="Switzerland", aliases=swi_locus).set_output(transform='pandas')),
    ('swe', CountryRecoder(country_name="Sweden", aliases=swe_locus).set_output(transform='pandas')),
    ('nor', CountryRecoder(country_name="Norway", aliases=nor_locus).set_output(transform='pandas')),
    ('den', CountryRecoder(country_name="Denmark", aliases=den_locus).set_output(transform='pandas')),
    ('fin', CountryRecoder(country_name="Finland", aliases=fin_locus).set_output(transform='pandas')),
    ('ice', CountryRecoder(country_name="Iceland", aliases=ice_locus).set_output(transform='pandas')),
    ('jpn', CountryRecoder(country_name="Japan", aliases=jpn_locus).set_output(transform='pandas')),
    ('tai', CountryRecoder(country_name="Taiwan", aliases=tai_locus).set_output(transform='pandas')),
    ('hun', CountryRecoder(country_name="Hungary", aliases=hun_locus).set_output(transform='pandas')),
    ('rus', CountryRecoder(country_name="Russia", aliases=rus_locus).set_output(transform='pandas')),
    ('sin', CountryRecoder(country_name="Singapore", aliases=sin_locus).set_output(transform='pandas')),
    ('skr', CountryRecoder(country_name="South Korea", aliases=skr_locus).set_output(transform='pandas')),
    ('cz', CountryRecoder(country_name="Czech Republic", aliases=cz_locus).set_output(transform='pandas')),
    ('chi', CountryRecoder(country_name="Chile", aliases=chi_locus).set_output(transform='pandas')),
    ('cyp', CountryRecoder(country_name="Cyprus", aliases=cyp_locus).set_output(transform='pandas')),
    ('isr', CountryRecoder(country_name='Israler', aliases=isr_locus).set_output(transform='pandas')),
    ('tha', CountryRecoder(country_name='Thailand', aliases=tha_locus).set_output(transform='pandas')),
    ('blr', CountryRecoder(country_name='Belarus', aliases=blr_locus).set_output(transform='pandas')),
    ('est', CountryRecoder(country_name='Estonia', aliases=est_locus).set_output(transform='pandas')),
    ('col', CountryRecoder(country_name='Colombia', aliases=col_locus).set_output(transform='pandas')),
    ('mex', CountryRecoder(country_name='Mexico', aliases=mex_locus).set_output(transform='pandas')),
    ('ukr', CountryRecoder(country_name='Ukraine', aliases=ukr_locus).set_output(transform='pandas')),
    ('rom', CountryRecoder(country_name='Romania', aliases=rom_locus).set_output(transform='pandas')),
    ('ksr', CountryRecoder(country_name='Saudi Arabia', aliases=ksa_locus).set_output(transform='pandas')),
    ('pak', CountryRecoder(country_name='Pakistan', aliases=pak_locus).set_output(transform='pandas')),
    ('mly', CountryRecoder(country_name='Malaysia', aliases=mly_locus).set_output(transform='pandas')),
    ('qat', CountryRecoder(country_name='Qatar', aliases=qat_locus).set_output(transform='pandas')),
    ('uae', CountryRecoder(country_name='United Arab Emirates', aliases=uae_locus).set_output(transform='pandas')),
    ('per', CountryRecoder(country_name='Peru', aliases=per_locus).set_output(transform='pandas')),
    ('jam', CountryRecoder(country_name='Jamaica', aliases=jam_locus).set_output(transform='pandas')),
    ('slk', CountryRecoder(country_name='Slovakia', aliases=slk_locus).set_output(transform='pandas')),
    ('slv', CountryRecoder(country_name='Slovenia', aliases=slv_locus).set_output(transform='pandas')),
    ('cro', CountryRecoder(country_name='Croatia', aliases=cro_locus).set_output(transform='pandas')),
    ('egy', CountryRecoder(country_name='Egypt', aliases=egy_locus).set_output(transform='pandas')),
    ('rsa', CountryRecoder(country_name='South Africa', aliases=rsa_locus).set_output(transform='pandas')),
    ('srb', CountryRecoder(country_name='Serbia', aliases=srb_locus).set_output(transform='pandas')),
    ('ban', CountryRecoder(country_name='Bangladesh', aliases=ban_locus).set_output(transform='pandas')),
    ('dom', CountryRecoder(country_name='Dominican Republic', aliases=dom_locus).set_output(transform='pandas')),
    ('lth', CountryRecoder(country_name='Lithuania', aliases=lth_locus).set_output(transform='pandas')),
    ('gha', CountryRecoder(country_name='Ghana', aliases=gha_locus).set_output(transform='pandas')),
    ('nig', CountryRecoder(country_name='Nigeria', aliases=nig_locus).set_output(transform='pandas')),
    ('eth', CountryRecoder(country_name='Ethiopia', aliases=eth_locus).set_output(transform='pandas')),
    ('tur', CountryRecoder(country_name='Turkey', aliases=tur_locus).set_output(transform='pandas')),
    ('pan', CountryRecoder(country_name='Panama', aliases=pan_locus).set_output(transform='pandas')),
    ('irn', CountryRecoder(country_name='Iran', aliases=irn_locus).set_output(transform='pandas')),
    ('ken', CountryRecoder(country_name='Kenya', aliases=ken_locus).set_output(transform='pandas')),
    ('lux', CountryRecoder(country_name='Luxembourg', aliases=lux_locus).set_output(transform='pandas')),
    ('lie', CountryRecoder(country_name='Liechtenstein', aliases=lie_locus).set_output(transform='pandas')),
    ('arg', CountryRecoder(country_name='Argentina', aliases=arg_locus).set_output(transform='pandas')),
    ('uru', CountryRecoder(country_name='Uruguay', aliases=uru_locus).set_output(transform='pandas')),
    ('kuw', CountryRecoder(country_name='Kuwait', aliases=kuw_locus).set_output(transform='pandas')),
    ('nam', CountryRecoder(country_name='Namibia', aliases=nam_locus).set_output(transform='pandas')),
    ('tan', CountryRecoder(country_name='Tanzania', aliases=tan_locus).set_output(transform='pandas')),
    ('bos', CountryRecoder(country_name='Bosnia and Herzegovina', aliases=bos_locus).set_output(transform='pandas')),
    ('sle', CountryRecoder(country_name='Sierra Leone', aliases=sle_locus).set_output(transform='pandas')),
    ('ind', CountryRecoder(country_name='India', aliases=ind_locus).set_output(transform='pandas')),
    ('uga', CountryRecoder(country_name='Uganda', aliases=uga_locus).set_output(transform='pandas')),
    ('cam', CountryRecoder(country_name='Cameroon', aliases=cam_locus).set_output(transform='pandas')),
    ('rwa', CountryRecoder(country_name='Rwanda', aliases=rwa_locus).set_output(transform='pandas')),
    ('zim', CountryRecoder(country_name='Zimbabwe', aliases=zim_locus).set_output(transform='pandas')),
    ('bar', CountryRecoder(country_name='Barbados', aliases=bar_locus).set_output(transform='pandas')),
    ('irq', CountryRecoder(country_name='Iraq', aliases=irq_locus).set_output(transform='pandas')),
    ('nep', CountryRecoder(country_name='Nepal', aliases=nep_locus).set_output(transform='pandas')),
    ('sen', CountryRecoder(country_name='Senegal', aliases=sen_locus).set_output(transform='pandas')),
])

country_trnsf = ColumnTransformer([
    ('country', country_pipe, ['Affiliations', 'Country'])
], remainder='passthrough')

country_trnsf.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('country',
                                 Pipeline(steps=[('usa',
                                                  CountryRecoder(aliases=['USA',
                                                                          'United '
                                                                          'States',
                                                                          'US',
                                                                          'Alabama',
                                                                          'California',
                                                                          'Albuquerque',
                                                                          'Ames '
                                                                          'IA',
                                                                          'Amherst '
                                                                          'MA',
                                                                          'Jackson '
                                                                          'MS',
                                                                          'Indianapolis',
                                                                          'Jacksonville '
                                                                          'FL',
                                                                          'Ann '
                                                                          'Arbor',
                                                                          'Albert '
                                                                          'Einstein '
                                                                          'College',
                                                                          'American '
                                                                          'Cancer '
                                                                          'Society',
                                                                          'Atlanta',
                                                                          'Massachusetts',
                                                                          'Ma...
                                                  CountryRecoder(aliases=['South '
                                                                          'Korea',
                                                                          'Republic '
                                                                          'of '
                                                                          'Korea',
                                                                          'Busan',
                                                                          'Seoul',
                                                                          'Cheongjusi',
                                                                          'Chungnam',
                                                                          'Daegu',
                                                                          'Daejeon',
                                                                          'Goyangsi',
                                                                          'Gwangju',
                                                                          'Gyeonggido',
                                                                          'Ulsan',
                                                                          'Hwasun',
                                                                          'Korea '
                                                                          'University',
                                                              

In [7]:
df_count = country_trnsf.fit_transform(df_semi)
df_count.columns = recover_columns_names(df_count)

In [8]:
nocountry = df_count[df_count['Country'] == ''].reset_index(drop=True)
nocountry

,Affiliations,Country,PubMed ID,Author
0,"Division of Rheumatology, Department of Medici...",,38281467,Daniel Wallace
1,,,37997724,G Y Heo
2,Naseri & Associates Public Health Consultancy ...,,37802043,Take Naseri
3,Human Genetics Center,,37777527,Megan L Grove
4,"Department of Epidemiology, Human Genetics, E...",,37777527,Megan L Grove
5,"Experimental Oncology, Molecular Oncology Pro...",,37306424,Carmen Guerra
6,"Department of Molecular Biology, Medical Bioc...",,36977412,Larissa Milano
7,"Department of Molecular Biology, Medical Bioc...",,36977412,Jean-Yves Masson
8,Division of Pulmonary Sciences Critical Care M...,,36477530,David A Schwartz
9,"Human Genetics Center, Department of Epidemiol...",,35264566,Paul S de Vries
